## PART 1: 

#### HOW TO CLEAN DATA WITH PYTHON

**Cleaning US Census Data**

You just got hired as a Data Analyst at the Census Bureau, which collects census data and creates interesting visualizations and insights from it.

The person who had your job before you left you all the data they had for the most recent census. It is in multiple csv files. They didn’t use pandas, they would just look through these csv files manually whenever they wanted to find something. Sometimes they would copy and paste certain numbers into Excel to make charts.

The thought of it makes you shiver. This is not scalable or repeatable.

Your boss wants you to make some scatterplots and histograms by the end of the day. Can you get this data into pandas and into reasonable shape so that you can make these histograms?


**Inspect the Data!**

**1.** The first visualization your boss wants you to make is a scatterplot that shows average income in a state vs proportion of women in that state.

Open some of the census csv files in the navigator. How are they named? What kind of information do they hold? Will they help us make this graph?


In [ ]:
import pandas as pd
import glob
import matplotlib.pyplot as plt

**2.** It will be easier to inspect this data once we have it in a DataFrame. You can’t even call .head() on these csvs! How are you supposed to read them?
   Using glob, loop through the census files available and load them into DataFrames. Then, concatenate all of those DataFrames together into one DataFrame, called something like us_census.

In [ ]:
files = glob.glob("states*.csv")

df_list = [pd.read_csv(fname, index_col=0) for fname in files]  
us_census = pd.concat(df_list, ignore_index=True)                  

us_census.head()

**3.** Look at the .columns and the .dtypes of the us_census DataFrame. Are those datatypes going to hinder you as you try to make histograms?

In [ ]:
# we can also get above required output with 'info()' function
# us_census.info()

print(us_census.dtypes)
print(us_census.columns)

**4.** Look at the .head() of the DataFrame so that you can understand why some of these dtypes are objects instead of integers or floats.
Start to make a plan for how to convert these columns into the right types for manipulation.

In [ ]:
us_census.head()

**Regex to the Rescue**

**5.** Use regex to turn the Income column into a format that is ready for conversion into a numerical type.

In [ ]:
# First method to achieve the above stated goal
#split_df = us_census['Income'].str.split('$', expand= True)
#us_census['Income'] =  pd.to_numeric(split_df[1])

# 2nd method

replace_df = us_census.Income.replace('[$,]','',regex=True)
us_census['Income'] =  pd.to_numeric(replace_df)
#print(us_census.head())
#print(us_census.dtypes)
us_census['Income'].head()

**6.** Look at the GenderPop column. We are going to want to separate this into two columns, the Men column, and the Women column.
Split the column into those two new columns using str.split and separating out those results.

In [ ]:
sep_genders = us_census['GenderPop'].str.split('_', expand=True)
print(sep_genders.head())
us_census['Men'] = sep_genders[0]
us_census['Women'] = sep_genders[1]

us_census.head()

**7.** Convert both of the columns into numerical datatypes.
There is still an M or an F character in each entry! We should remove those before we convert.

In [ ]:
us_census['Men'] = pd.to_numeric(us_census['Men'].apply(lambda x:x[:-1]))
us_census['Women'] = pd.to_numeric(us_census['Women'].apply(lambda x:x[:-1]))

us_census.head()

**8.** Now you should have the columns you need to make the graph and make sure your boss does not slam a ruler angrily on your desk because you’ve wasted your whole day cleaning your data with no results to show!

    Use matplotlib to make a scatterplot!

    plt.scatter(the_women_column, the_income_column) 
    Remember to call plt.show() to see the graph!

In [ ]:
plt.scatter(us_census.Women, us_census.Income)
plt.xlabel('Female Population')
plt.ylabel('Income')
plt.title('Female Population and Income')
plt.show()

**9.** Did you get an error? These monstrous csv files probably have nan values in them! Print out your column with the number of women per state to see.

We can fill in those nans by using pandas’ .fillna() function.

You have the TotalPop per state, and you have the Men per state. As an estimate for the nan values in the Women column, you could use the TotalPop of that state minus the Men for that state.

Print out the Women column after filling the nan values to see if it worked!

In [ ]:
print(us_census['Women'])
us_census['Women'] = us_census['Women'].fillna(us_census.TotalPop - us_census.Men)
us_census['Women'] = us_census['Women'].astype(int)     # to convert from float to int
print(us_census['Women'])

**10.** We forgot to check for duplicates! Use .duplicated() on your census DataFrame to see if we have duplicate rows in there.

In [ ]:
print(us_census.duplicated('State'))
print(us_census.duplicated().value_counts())
us_census[us_census.duplicated('State')]

**11.** Drop those duplicates using the .drop_duplicates() function.

In [ ]:
print(us_census.drop_duplicates())
us_census = us_census.drop_duplicates(ignore_index=True)

us_census.head(15)

**12.** Make the scatterplot again. Now, it should be perfect! Your job is secure, for now.

In [ ]:
plt.scatter(us_census.Women, us_census.Income)
plt.xlabel('Female Population')
plt.ylabel('Income')
plt.title('Female Population and Income')
plt.show()

### Histograms of Races

**13.** Now, your boss wants you to make a bunch of histograms out of the race data that you have. Look at the .columns again to see what the race categories are.


In [ ]:
us_census.columns

**14.** Try to make a histogram for each one!

You will have to get the columns into numerical format, and those percentage signs will have to go.

Don’t forget to fill the nan values with something that makes sense! You probably dropped the duplicate rows when making your last graph, but it couldn’t hurt to check for duplicates again.

In [ ]:
us_census.Hispanic = us_census.Hispanic.replace('[%,]','',regex=True)
us_census.Hispanic = pd.to_numeric(us_census.Hispanic)

us_census.White = us_census.White.replace('[%,]','',regex=True)
us_census.White = pd.to_numeric(us_census.White)

us_census.Black = us_census.Black.replace('[%,]','',regex=True)
us_census.Black = pd.to_numeric(us_census.Black)

us_census.Native = us_census.Native.replace('[%,]','',regex=True)
us_census.Native = pd.to_numeric(us_census.Native)

us_census.Asian = us_census.Asian.replace('[%,]','',regex=True)
us_census.Asian = pd.to_numeric(us_census.Asian)

us_census.Pacific = us_census.Pacific.replace('[%,]','',regex=True)
us_census.Pacific = pd.to_numeric(us_census.Pacific)

print(us_census.dtypes)

In [ ]:
# to check for missing values values before plotting
print(us_census.Hispanic.isna().value_counts())
print(us_census.White.isna().value_counts())
print(us_census.Black.isna().value_counts())
print(us_census.Native.isna().value_counts())
print(us_census.Asian.isna().value_counts())
print(us_census.Pacific.isna().value_counts())

In [ ]:
# Filling missing values 
us_census['Pacific'] = us_census['Pacific'].fillna(100 - (us_census['Hispanic'] +us_census['Black']+us_census['White']+us_census['Native']+us_census['Asian']  ))

print(us_census.Pacific.isna().value_counts())

In [ ]:
plt.hist(us_census.Hispanic, bins=15, label= "hispanic")
plt.hist(us_census.Black, bins=15,label= "black")
plt.hist(us_census.White, bins=15,label= "white")
plt.hist(us_census.Native, bins=15,label= "native")
plt.hist(us_census.Asian, bins=15,label= "asian")
plt.hist(us_census.Pacific, bins=15, label= "pacific")
plt.legend()
plt.show()

# PART 2:

### LEARN DATA ANALYSIS WITH PANDAS

**Petal Power Inventory**
You’re the lead data analyst for a chain of gardening stores called Petal Power. Help them analyze their inventory!

**Answer Customer Emails**

**1.** Data for all of the locations of Petal Power is in the file inventory.csv. Load the data into a DataFrame called inventory.


In [ ]:
inventory = pd.read_csv("inventory.csv")
inventory

**2.** Inspect the first 10 rows of inventory.

In [ ]:
inventory.head(10)

**3.** The first 10 rows represent data from your Staten Island location. Select these rows and save them to staten_island.

In [ ]:
staten_island = inventory.head(10)

**4.** A customer just emailed you asking what products are sold at your Staten Island location. Select the column product_description from staten_island and save it to the variable product_request.


In [ ]:
product_request = staten_island.product_description     # another way is:  staten_island['product_description']
product_request

**5.** Another customer emails to ask what types of seeds are sold at the Brooklyn location. Select all rows where location is equal to Brooklyn and product_type is equal to seeds and save them to the variable seed_request

In [ ]:
seed_request = inventory[(inventory.location == 'Brooklyn') & (inventory.product_type == 'seeds')]
seed_request

**Inventory**

**6.** Add a column to inventory called in_stock which is True if quantity is greater than 0 and False if quantity equals 0.

In [ ]:
in_stock = lambda x: True if x > 0 else False
inventory['in_stock'] = inventory['quantity'].apply(in_stock)  # we could also use 'map' function

inventory

**7.** Petal Power wants to know how valuable their current inventory is.Create a column called total_value that is equal to price multiplied by quantity.

In [ ]:
#inventory['total_value'] = inventory['price'] * inventory['quantity']     // This is one way

#inventory['total_value'] = inventory['price'].multiply(inventory['quantity']) // another way
# Below is through lambda function

total_value = lambda x: x.price * x.quantity

inventory['total_value'] = inventory.apply(total_value, axis = 1)
inventory

**8.** The Marketing department wants a complete description of each product for their catalog. The following lambda function combines product_type and product_description into a single string:

combine_lambda = lambda row: \
    '{} - {}'.format(row.product_type,
                     row.product_description)

In [ ]:
combine_lambda = lambda row: '{} - {}'.format(row.product_type,row.product_description)
combine_lambda

**9.** Using combine_lambda, create a new column in inventory called full_description that has the complete description of each product.


In [ ]:
inventory['full_description'] = inventory.apply(combine_lambda, axis = 1)
inventory